In [214]:
import os, pprint

template_api_path = "../sample_files/base-template/"
source_api_path = "../sample_files/api-current/"
destination_api_path = "../sample_files/api-destination/"

filename = "build.gradle"
template_file = os.path.join(template_api_path, filename)
source_file = os.path.join(source_api_path, filename)

In [215]:
with open(template_file) as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
    
# print(lines)

In [216]:
# with open(template_file) as file:
#     for line in file:
#         print(line.rstrip())

In [217]:
with open(template_file) as fp:
    line = fp.readline()
    cnt = 1
    while line:
#         print("Line {}: {}".format(cnt, line.strip()))
        line = fp.readline()
        cnt += 1

In [221]:
with open(template_file) as f:
    line_start = 0
    line_end = 0
    for index, line in enumerate(f):
        if line.startswith("dependencies {"):
            line_start = index
        if line_start !=0 and line.startswith("}"):
            line_end = index
            break
            
with open(template_file) as f:
    i = 0
    template_list = []
    for index, line in enumerate(f):
        if index > line_start and index < line_end and line !='\n':
            rep_dic = {}
            tmp_line = line.strip().replace("'", "").replace("ion ", "ion:").replace(" ", "")
            print("Line {}: {}".format(index, tmp_line))
            i += 1
            if i == 1 or i == 8 or i >= 13:
                temp = tmp_line.split(":")
                if len(temp) > 1:
                    ver_list = temp[len(temp)-1].split('.')
                    ver_list = [int(x) for x in ver_list]
                    
                    rep_dic["prefix"] = temp[0]
                    rep_dic["artifact"] = temp[len(temp)-2]
                    rep_dic["version"] = temp[len(temp)-1]
                    rep_dic["version_sum"] = sum(ver_list)
                    rep_dic['template_line'] = line
                    rep_dic['rule'] = 1
#                     print(rep_dic)
            elif i == 10:
                temp = tmp_line.replace(",", ":").split(":")
                if len(temp) > 1:
                    ver_list = temp[len(temp)-1].split('.')
                    ver_list = [int(x) for x in ver_list]
                    
                    rep_dic["prefix"] = temp[0]
                    rep_dic["artifact"] = temp[len(temp)-3]
                    rep_dic["version"] = temp[len(temp)-1]
                    rep_dic["version_sum"] = sum(ver_list)
                    rep_dic['template_line'] = line
                    rep_dic['rule'] = 2
#                     print(rep_dic)
            else:
                temp = tmp_line.split(":")
                if len(temp) > 1:
                    rep_dic["prefix"] = temp[0]
                    rep_dic["artifact"] = temp[len(temp)-1]
                    rep_dic["version"] = None
                    rep_dic["version_sum"] = 0
                    rep_dic['template_line'] = line
                    rep_dic['rule'] = None
            template_list.append(rep_dic)
#             print(rep_dic)
                
pprint.pprint(template_list)

Line 45: implementation:com.barclays.api:api-core-boot:2.0.21
Line 46: implementation:org.springframework.cloud:spring-cloud-config-server
Line 47: implementation:org.springframework.boot:spring-boot-starter-thymeleaf
Line 48: implementation:org.springframework.cloud:spring-cloud-starter-vault-config
Line 49: implementation:org.springframework.cloud:spring-cloud-starter-bus-amqp
Line 50: implementation:org.springframework.cloud:spring-cloud-config-monitor
Line 51: implementation:org.springframework.cloud:spring-cloud-starter-netflix-eureka-client
Line 52: implementation:org.bitbucket.b_c:jose4j:0.7.3
Line 53: testImplementation:org.springframework.boot:spring-boot-starter-test
Line 54: testImplementation:group:com.github.tomakehurst,name:wiremock,version:2.26.3
Line 56: testImplementation:junit:junit
Line 57: testImplementation:org.mockito:mockito-core
Line 58: testImplementation:org.raml.raml-parser-2:1.0.13
Line 60: testImplementation:com.intuit.karate:karate-junit5:0.8.0.1
Line 61: 

In [274]:
from ordered_set import OrderedSet

with open(template_file, "r") as f, open(source_file, "r+") as out:
    line_start = 0
    line_end = 0
    for index, line in enumerate(f):
        if line.startswith("dependencies {"):
            line_start = index
        if line_start !=0 and line.startswith("}"):
            line_end = index
            break
            
    line_start_out = 0
    line_end_out = 0
    for index_out, line_out in enumerate(out):
        if line_out.startswith("dependencies {"):
            line_start_out = index_out
        if line_start_out !=0 and line_out.startswith("}"):
            line_end_out = index_out
            break

with open(template_file) as f, open(source_file, "r") as out:
    for index, line in enumerate(f):
        if index > line_start and index < line_end:
            temp_in = line.strip().replace("'", "").replace("tion ", "tion:").replace(" ", "").split(":")
#             print("Line {}: {}".format(index, line.strip()))
#             print("Line {}: {}".format(index, temp_in))
            
    print("\n*******============*******\n")
    
    template_set = OrderedSet()
    unchanged_set = OrderedSet()
    changed_set = OrderedSet()
    for index_out, line_out in enumerate(out):
        if index_out > line_start_out and index_out < line_end_out:
#             print("Line {}: {}".format(index_out, line_out.strip()))
            for l in template_list:
                if l['artifact'] in line_out:
                    if l['rule'] == 1:
                        tmp_line = line_out.strip().replace("'", "").replace("ion ", "ion:").replace(" ", "")
                        temp = tmp_line.split(":")
                        if len(temp) > 1:
                            ver_list = temp[len(temp)-1].split('.')
                            ver_list = [int(x) for x in ver_list]

                            prefix = temp[0]
                            version = temp[len(temp)-1]
                            version_sum = sum(ver_list)
                            template_line = line
                        
                            if version_sum < l['version_sum']:
                                line_out = line_out.replace(version, l['version'])
                                template_set.add(l['template_line'])
                                changed_set.add(line_out)
                            elif version_sum > l['version_sum']:
                                template_set.add(line_out)
                                changed_set.add(l['template_line'])
                            else:
                                template_set.add(l['template_line'])
# #                                 changed_set.add(line_out)
                    elif l['rule'] == 2:
                        tmp_line = line_out.strip().replace("'", "").replace("ion ", "ion:").replace(" ", "")
                        temp = tmp_line.replace(",", ":").split(":")
                        if len(temp) > 1:
                            ver_list = temp[len(temp)-1].split('.')
                            ver_list = [int(x) for x in ver_list]

                            prefix = temp[0]
                            version = temp[len(temp)-1]
                            version_sum = sum(ver_list)
                            template_line = line
                        
                            if version_sum < l['version_sum']:
                                line_out = line_out.replace(version, l['version'])
                                template_set.add(l['template_line'])
                                changed_set.add(line_out)
                            elif version_sum > l['version_sum']:
                                template_set.add(line_out)
                                changed_set.add(l['template_line'])
                            else:
                                template_set.add(l['template_line'])
#                                 changed_set.add(line_out)
                    else:
                        template_set.add(l['template_line'])
#                         changed_set.add(line_out)
                else:
                    unchanged_set.add(line_out)

print("\nLIST-3*******============*******\n")
pprint.pprint(lst3)
print("\n*******============*******\n")
# diff1 = (template_set - unchanged_set)
# diff2 = (unchanged_set - changed_set)
diff1 = (template_set - changed_set)
diff2 = (unchanged_set + diff1)
t = []
t.append({'dependencies{'})
t.append(diff1)
t.append(diff2)
t.append({'}'})
print("\n*******============*******\n")
dep_list = [i for sub in t for i in sub]
pprint.pprint(dep_list)


*******============*******


LIST-3*******============*******

OrderedSet(["\tcompile 'com.barclays.api:api-core-boot:2.0.21'\n", "\ttestImplementation 'com.intuit.karate:karate-junit5:0.8.0.1'\n", "\ttestImplementation 'com.intuit.karate:karate-apache:0.8.0.1'\n"])

*******============*******



TypeError: unsupported operand type(s) for +: 'OrderedSet' and 'OrderedSet'

In [271]:
for line in dep_list:
    print(line)

dependencies{
	implementation 'com.barclays.api:api-core-boot:2.0.21'

	implementation 'org.springframework.cloud:spring-cloud-config-server'

	implementation 'org.springframework.boot:spring-boot-starter-thymeleaf'

	implementation 'org.springframework.cloud:spring-cloud-starter-vault-config'

	implementation 'org.springframework.cloud:spring-cloud-starter-bus-amqp'

	implementation 'org.springframework.cloud:spring-cloud-config-monitor'

	implementation 'org.springframework.cloud:spring-cloud-starter-netflix-eureka-client'

	implementation 'org.bitbucket.b_c:jose4j:0.7.3'

	testImplementation 'org.springframework.boot:spring-boot-starter-test'

	testImplementation group: 'com.github.tomakehurst', name: 'wiremock', version: '2.26.3'

	testImplementation 'junit:junit'

	testImplementation 'org.mockito:mockito-core'

	testImplementation 'org.raml.raml-parser-2:1.0.13'

	testCompile 'com.intuit.karate:karate-junit5:0.9.6'

	testCompile 'com.intuit.karate:karate-apache:0.9.6'

	compile 'c